In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
from random import randint

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split

In [48]:
# Read the images and labels of gemstones from training data
def extract_labels(name):
    parts = name.split('/')
    train = parts[1]
    if 'Fake' in parts[2]:
        label = 'fake'
    else:
        label = 'real'
    return train, label
    

def read_images():
    Images = []
    for root, dirs, files in os.walk('datasets'):
        f = os.path.basename(root)
        for file in files:
            try:
                image = cv2.imread(root+'/'+file)
                image = np.array(image)
                train, label = extract_labels(root)
                Images.append([train, image, label])
            except Exception as e:
                print(e)
    return Images  

In [ ]:
images = read_images()